In [1]:
# import packages
import numpy as np
import pandas as pd
import json

In [2]:
# unit conversions
def convert_TF_to_TK(TF):
    return (5/9)*(TF-32)+273.15

def convert_Ppsi_to_Pbar(Ppsi):
    return Ppsi * 0.0689475729

def convert_ft_to_m(zft):
    return zft * 0.3048

In [3]:
# set workbook and sheet name
wb_name = 'ADEPT(2023-01-17).xlsm'
sh_name = 'Fluid-LUT'

#tags : ('depth', 'depth_ft', 'P_psig', 'T_F', 'yAsp_L1', 'yAsp_L2', beta_V, )

# get tags
df_tags = pd.read_excel(wb_name, sheet_name=sh_name, nrows=2)

# get numerical data
df = pd.read_excel(wb_name, sheet_name=sh_name, header=2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 37 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   -             200 non-null    float64
 1   ft            200 non-null    float64
 2   psig          200 non-null    float64
 3   °F            200 non-null    float64
 4   g/cc          200 non-null    int64  
 5   cP            200 non-null    int64  
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   gAsp_L1/gAsp  200 non-null    float64
 9   wtfrac        200 non-null    float64
 10  Unnamed: 10   200 non-null    float64
 11  wt%           200 non-null    int64  
 12  Unnamed: 12   200 non-null    float64
 13  Unnamed: 13   200 non-null    float64
 14  g/cc.1        200 non-null    int64  
 15  Unnamed: 15   200 non-null    float64
 16  Unnamed: 16   200 non-null    float64
 17  Unnamed: 17   200 non-null    float64
 18  MPa½          200 non-null    

In [4]:
data = df.to_numpy()

In [6]:
# convert to standard units
nZ, nProp = data.shape
z = convert_ft_to_m(data[:,1])      #depth (ft to m)
P = convert_Ppsi_to_Pbar(data[:,2]) #pressure (psi to bar)
T = convert_TF_to_TK(data[:,3])     #temperature (F to K)
beta = data[:,11:14] / 100         #phase frac (wt% -> wtf)
dens = data[:,14:17] * 1000        #density (g/cc -> kg/m3)
dens_Asp = data[:,17] * 1000
visco = data[:,22:26] / 1000        #viscosity (cP -> Pa.s)

In [7]:
# volume fraction
volFrac = np.nan_to_num(beta/dens, copy=False)

sumFrac = np.sum(volFrac, axis=1)
volFrac = volFrac / sumFrac.reshape(volFrac.shape[0],1)

C:\Users\cjsis\AppData\Local\Temp\ipykernel_22044\1070966314.py:2: RuntimeWarning: invalid value encountered in true_divide
  volFrac = np.nan_to_num(beta/dens, copy=False)


In [8]:
# populate dict
my_dict = {
    'depth': z.tolist(), 
    'P': P.tolist(), 
    'T': T.tolist(),
    'Ceq': data[:,8].tolist(),
    'yAsp_V': np.zeros(nZ).tolist(),
    'yAsp_L1': data[:,9].tolist(),
    'yAsp_L2': data[:,10].tolist(),
    'wtFrac_V': beta[:,0].tolist(),
    'wtFrac_L1': beta[:,1].tolist(),
    'wtFrac_L2': beta[:,2].tolist(),
    'volFrac_V': volFrac[:,0].tolist(),
    'volFrac_L1': volFrac[:,1].tolist(),
    'volFrac_L2': volFrac[:,2].tolist(),
    'dens_V': dens[:,0].tolist(),
    'dens_L1': dens[:,1].tolist(),
    'dens_L2': dens[:,2].tolist(),
    'dens_Asp': dens_Asp.tolist(),
    'SP_V': data[:,18].tolist(),
    'SP_L1': data[:,19].tolist(),
    'SP_L2': data[:,20].tolist(),
    'SP_Asp': data[:,21].tolist(),
    'visco_V': visco[:,0].tolist(),
    'visco_L1': visco[:,1].tolist(),
    'visco_L2': visco[:,2].tolist(),
    'visco_Asp': np.zeros(nZ).tolist()
}

In [ ]:
depth = my_dict['depth']
dens_L1 = my_dict['dens_L1']
visco_L1 = my_dict['visco_L1']

In [ ]:
# save to json
json_name = 'temp_json.json'
with open(json_name, 'w') as json_file:
    json.dump(my_dict, json_file)